In [41]:
import numpy as np
import torch.nn as nn
from sklearn.linear_model import LinearRegression
from src.helpers.functions import get_data, pc_T, predict_pca, estimate_AR_res, generate_data
from src.helpers.functions import select_AR_lag_SIC, winsor, lag_matrix
from src.helpers.refactored import ar_forecast, scale_X, reduce_dims, loocv_ts, standardize
from src.helpers.autoencoder import Autoencoder

In [43]:
def out_sample(X, y, dim_method="ae"):
    """ Function to perform out of sample forecasting """
    h = 1
    T = y.shape[0]

    M = (1984-1959)*12  # In sample periods
    N = T - M  # Out of sample periods

    forecast_pca = np.zeros((N, 1))  # Forecast errors of PCA 
    forecast_spca = np.zeros((N, 1))  # Forecast errors of scaled PCA
    forecast_ar = np.zeros((N, 1))  # Forecast errors of AR model
    actual_y = np.zeros((N, 1))  # Actual values of y

    # Initialize the regression models
    reg_pc = LinearRegression()
    reg_spc = LinearRegression()

    p_max = 1  # Max number of lags for AR(p) model
    nfac =  10  # Number of factors to reduce to
    scale_method = "regression"
    
    if dim_method == "ae":
        # Initialize the autoencoder
        ae = Autoencoder(input_dim=X.shape[1], hidden_dim=nfac, activation=nn.ReLU, layer_dims=[32, 32, 32])

        # Train the autoencoder on the in sample data
        ae.train(standardize(X[:M, :]))

    # Loop over all out of sample periods
    for n in range(N):
        #print("Forecast {} out of {}".format(n, N))

        # Use all available data up to time t
        X_t = X[:(M + n), :]
        y_t = y[:M + n]
        actual_y[n] = y[M + n]

        # Standardize the data
        X_t = (X_t - np.mean(X_t, axis=0)) / np.std(X_t, axis=0)

        # Get number lags
        p_AR_star_n = select_AR_lag_SIC(y_t, h, p_max=p_max)

        # Compute the forecast of the AR model
        forecast_ar[n] = ar_forecast(p_AR_star_n, y_t, h)
        
        #### STEP 1: Scaling factors ####

        # Compute the betas for scaling the variables
        beta = scale_X(X_t, y_t, h, method=scale_method, p_AR_star_n=p_AR_star_n)
        
        # Winsorizing the betas
        beta_win = winsor(np.abs(beta), p=(0, 90))

        # Scale the factors by the winsorized betas
        scaleX_t = X_t * beta_win

        #### Intermezzo: Find the optimal number of factors ####
        
        #if n == 0:
        #    nfac = loocv_ts(X=X_t, y=y_t, h=h, p_AR_star_n=p_AR_star_n, method="pca")
        
        #### STEP 2: Dimension Reduction ####            

        # Compute the reduced dimensionality representation of the factors
        #x_pc = reduce_dims(X=X_t, nfac=nfac, method="pca")
        x_spc = reduce_dims(X=scaleX_t, nfac=nfac, method=dim_method, dim_red_model=ae)

        #### STEP 3: Forecasting ####

        # Add lag of y_t to the factors
        if p_AR_star_n > 0:
            #x_pc = lag_matrix(x_pc, y_t, p_AR_star_n)
            x_spc = lag_matrix(x_spc, y_t, p_AR_star_n)
            y_t = y_t[p_AR_star_n-1:]
            
        # Estimate regression coefficients
        #reg_pc.fit(x_pc[:-h], y_t[h:])
        reg_spc.fit(x_spc[:-h], y_t[h:])
        
        # Compute the forecast of the PCA and scaled PCA model
        #forecast_pca[n] = reg_pc.predict(x_pc[-1].reshape(1, -1))
        forecast_spca[n] = reg_spc.predict(x_spc[-1].reshape(1, -1))

    # Compute the forecast errors
    error_pca = actual_y - forecast_pca
    error_spca = actual_y - forecast_spca
    error_ar = actual_y - forecast_ar
    
    # Compute the R squared out of sample against the AR model
    SSE_pca = np.sum(error_pca**2)
    SSE_spca = np.sum(error_spca**2)
    SSE_ar = np.sum(error_ar**2)

    R2_spca = (1 - SSE_spca / SSE_ar)
    R2_pca = (1 - SSE_pca / SSE_ar)

    print("R2_spca: ", round(R2_spca * 100, 2))
    print("R2_pca: ", round(R2_pca* 100, 2))
    
    return {"error_pca": error_pca,
            "error_spca": error_spca,
            "error_ar": error_ar,
            "R2_spca": R2_spca,
            "R2_pca": R2_pca}

In [32]:
variables = get_data()
X = variables['data'].values
y = variables['unemployment'].values

print("Shape of X: ", X.shape, "Shape of y: ", y.shape)

[*********************100%***********************]  1 of 1 completed
Shape of X:  (720, 123) Shape of y:  (720,)


In [44]:
# Run the forecasting exercise
for target in variables.keys():
    if target != 'inflation':
        continue
    print("Target: ", target)
    y = variables[target].values
    result = out_sample(X = X, y = y)

Target:  inflation
Epoch: 0, Loss: 0.7892279624938965
Epoch: 1, Loss: 0.7819177508354187
Epoch: 2, Loss: 0.776904284954071
Epoch: 3, Loss: 0.7724379301071167
Epoch: 4, Loss: 0.7661020159721375
Epoch: 5, Loss: 0.7565230131149292
Epoch: 6, Loss: 0.7447555065155029
Epoch: 7, Loss: 0.7285982370376587
Epoch: 8, Loss: 0.7078547477722168
Epoch: 9, Loss: 0.6831662654876709
Epoch: 10, Loss: 0.6626929640769958
Epoch: 11, Loss: 0.6441946029663086
Epoch: 12, Loss: 0.6326876878738403
Epoch: 13, Loss: 0.6213299632072449
Epoch: 14, Loss: 0.6141618490219116
Epoch: 15, Loss: 0.6053768992424011
Epoch: 16, Loss: 0.6153557896614075
Epoch: 17, Loss: 0.6012275218963623
Epoch: 18, Loss: 0.5899512767791748
Epoch: 19, Loss: 0.5891799926757812
Epoch: 20, Loss: 0.5828309655189514
Epoch: 21, Loss: 0.5773717164993286
Epoch: 22, Loss: 0.574475884437561
Epoch: 23, Loss: 0.5702841877937317
Epoch: 24, Loss: 0.5647417902946472
Epoch: 25, Loss: 0.5637141466140747
Epoch: 26, Loss: 0.5578098297119141
Epoch: 27, Loss: 0.55

#### Regression scaling, kpca RBF
Target:  inflation 
R2_spca:  12.39
R2_pca:  -0.76
Target:  unemployment
R2_spca:  17.22
R2_pca:  10.12
Target:  ip_growth
R2_spca:  13.22
R2_pca:  7.88
Target:  volatility
R2_spca:  5.4
R2_pca:  1.13

#### Distance correlation scaling, kpca RBF
Target:  inflation
R2_spca:  9.03
R2_pca:  -0.76
Target:  unemployment
R2_spca:  17.59
R2_pca:  10.12
Target:  ip_growth
R2_spca:  10.88
R2_pca:  7.88
Target:  volatility
R2_spca:  5.66
R2_pca:  1.13

#### Distance correlation scaling, kpca poly(5)
Target:  inflation
R2_spca:  9.73
R2_pca:  -0.76
Target:  unemployment
R2_spca:  18.86
R2_pca:  10.12
Target:  ip_growth
R2_spca:  12.63
R2_pca:  7.88
Target:  volatility
R2_spca:  4.63
R2_pca:  1.13




In [ ]:
errors_ar = result['error_ar']
errors_pca = result['error_pca']
errors_spca = result['error_spca']

#errors = pd.DataFrame({'errors_ar': errors_ar, 'errors_pca': errors_pca.flatten()})
#np.set_printoptions(formatter={'all':lambda x: str(x)[:7]})
np.set_printoptions(formatter={'float_kind':'{:f}'.format})
mse_ar = np.mean(errors_ar**2)
mse_pca = np.mean(errors_pca**2)
mse_spca = np.mean(errors_spca**2)

print("MSE AR: ", mse_ar)
print("MSE PCA: ", mse_pca)
print("MSE SPCA: ", mse_spca)